In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

### Funcion: describe_df

Esta función debe recibir como argumento un dataframe y debe devolver una dataframe como el de la imagen (no el de la imagen). Es decir un dataframe que tenga una columna por cada columan del dataframe original y como filas, los tipos de las columnas, el tanto por ciento de valores nulos o missings, los valores únicos y el porcentaje de cardinalidad. 

La figura muestra el resultado esperado de llamar a la función pedida con el dataset del titanic:
![image.png](attachment:image.png)

In [2]:
def describe_df(df):
    df_resultado = pd.DataFrame(columns=['DATA_TYPE', 'MISSINGS (%)', 'UNIQUE_VALUES', 'CARDIN (%)'], index=df.columns)
    total = len(df) 
    
    for column in df.columns:

        #Data type
        df_resultado.at[column, 'DATA_TYPE'] = df[column].dtype
        
        # Missings
        missing = df[column].isnull().sum()
        df_resultado.at[column, 'MISSINGS (%)'] = missing/total*100
        
        # Unique values
        df_resultado.at[column, 'UNIQUE_VALUES'] = df[column].nunique()

        # Cardinalidad
        cardinalidad = df[column].nunique()
        porcentaje_cardinalidad = (cardinalidad / total) * 100
        df_resultado.at[column, 'CARDIN (%)'] = round(porcentaje_cardinalidad, 2)

     
    return df_resultado.T

### Funcion: tipifica_variables

Esta función debe recibir como argumento un dataframe, un entero (`umbral_categoria`) y un float (`umbral_continua`). La función debe devolver un dataframe con dos columnas "nombre_variable", "tipo_sugerido" que tendrá tantas filas como columnas el dataframe. En cada fila irá el nombre de una de las columnas y una sugerencia del tipo de variable. Esta sugerencia se hará siguiendo las siguientes pautas:
+ Si la cardinalidad es 2, asignara "Binaria"
+ Si la cardinalidad es menor que `umbral_categoria` asignara "Categórica"
+ Si la cardinalidad es mayor o igual que `umbral_categoria`, entonces entra en juego el tercer argumento:
    * Si además el porcentaje de cardinalidad es superior o igual a `umbral_continua`, asigna "Numerica Continua"
    * En caso contrario, asigna "Numerica Discreta"

In [3]:
def tipifica_variables(df:pd.DataFrame, umbral_categoria:int, umbral_continua:float):
    """
    Clasifica las columnas de un DataFrame según su tipo sugerido basándose en cardinalidad y porcentaje de cardinalidad.
    Los tipos posibles son Binaria, Categórica, Numérica Continua y Numérica Discreta.

    Argumentos:
    df (pd.DataFrame): DataFrame con los datos cuyas variables se desean tipificar.
    umbral_categoria (int): Umbral de cardinalidad para determinar si una variable es categórica.
    umbral_continua (float): Umbral de porcentaje de cardinalidad para diferenciar entre numérica continua y discreta.

    Retorna:
    pd.DataFrame: DataFrame con dos columnas: 'nombre_variable' que contiene los nombres de las columnas originales,
                  y 'tipo_sugerido' que indica el tipo de dato sugerido para cada columna basado en su cardinalidad y 
                  porcentaje de cardinalidad.
    """
    resultado = pd.DataFrame()
    resultado['nombre_variable']=df.columns
    resultado['tipo_sugerido']=pd.Series()
    
    total_filas = len(df)
    
    for col in df.columns:
        cardinalidad = df[col].nunique()
        porcentaje_cardinalidad = (cardinalidad / total_filas) * 100
        
        if cardinalidad == 2:
            tipo_sugerido = 'Binaria'
        elif cardinalidad < umbral_categoria:
            tipo_sugerido = 'Categórica'
        else:
            if porcentaje_cardinalidad >= umbral_continua:
                tipo_sugerido = 'Numérica Continua'
            else:
                tipo_sugerido = 'Numérica Discreta'
        
        resultado.loc[resultado['nombre_variable'] == col, 'tipo_sugerido'] = tipo_sugerido
    
    return resultado

### Funcion: get_features_num_regression

Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de regresión, es decir debe ser una variable numérica continua o discreta pero con alta cardinalidad, además de un argumento 'umbral_corr', de tipo float que debe estar entre 0 y 1 y una variable float "pvalue" cuyo valor debe ser por defecto "None".

La función debe devolver una lista con las columnas numéricas del dataframe cuya correlación con la columna designada por "target_col" sea superior en valor absoluto al valor dado por "umbral_corr". Además si la variable "pvalue" es distinta de None, sólo devolvera las columnas numéricas cuya correlación supere el valor indicado y además supere el test de hipótesis con significación mayor o igual a 1-pvalue.

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable numérica continua del dataframe.

In [4]:
def get_features_num_regression(dataframe, target_col, umbral_corr, pvalue=None):
    # Comprobación de que dataframe es un DataFrame de pandas
    if not isinstance(dataframe, pd.DataFrame):
        print("Error: El argumento 'dataframe' debe ser un DataFrame de pandas.")
        return None
    
    # Comprobación de que target_col es una columna válida en dataframe
    if target_col not in dataframe.columns:
        print("Error: El argumento 'target_col' no es una columna válida en el DataFrame.")
        return None
    
    # Comprobación de que target_col es numérica
    if not np.issubdtype(dataframe[target_col].dtype, np.number):
        print("Error: El argumento 'target_col' debe ser una variable numérica continua.")
        return None
    
    # Comprobación de que umbral_corr está entre 0 y 1
    if not 0 <= umbral_corr <= 1:
        print("Error: El argumento 'umbral_corr' debe estar entre 0 y 1.")
        return None
    
    # Comprobación de que pvalue, si se proporciona, es un float entre 0 y 1
    if pvalue is not None:
        if not isinstance(pvalue, float) or not 0 <= pvalue <= 1:
            print("Error: El argumento 'pvalue' debe ser un número flotante entre 0 y 1.")
            return None
    
    # Calcular las correlaciones entre todas las características y target_col
    correlations = dataframe.corr()[target_col].abs()
    
    # Seleccionar las características con correlación superior a umbral_corr
    selected_features = correlations[correlations > umbral_corr].index.tolist()
    
    # Si se proporciona un pvalue, realizar un test de hipótesis para cada característica seleccionada
    if pvalue is not None:
        p_values = []
        for feature in selected_features:
            # Calcular el p-value utilizando una prueba de correlación de Pearson
            p_val = pearsonr(dataframe[feature], dataframe[target_col])[1]
            # Si el p-value es menor o igual a 1-pvalue, conservar la característica
            if p_val <= 1 - pvalue:
                p_values.append(p_val)
            else:
                selected_features.remove(feature)
        
        print("P-values:", p_values)
    
    return selected_features

### Funcion: get_features_cat_regression

Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de regresión, es decir debe ser una variable numérica continua o discreta pero con alta cardinalidad y una variable float "pvalue" cuyo valor por defecto será 0.05.

La función debe devolver una lista con las columnas categóricas del dataframe cuyo test de relación con la columna designada por 'target_col' supere en confianza estadística el test de relación que sea necesario hacer (es decir la función debe poder escoger cuál de los dos test que hemos aprendido tiene que hacer).

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable numérica continua del dataframe.

In [5]:
def get_features_cat_regression(df:pd.DataFrame, target_col:str, pvalue:float=0.05):
    """
    Identifica columnas categóricas en un DataFrame que están significativamente relacionadas con
    una columna objetivo numérica, utilizando el análisis de varianza (ANOVA).

    Argumentos:
    df (pd.DataFrame): DataFrame que contiene los datos a analizar.
    target_col (str): Nombre de la columna objetivo que debe ser numérica.
    pvalue (float, opcional): Umbral de p-valor para considerar significativas las columnas categóricas.

    Retorna:
    List[str]: Lista de nombres de columnas categóricas que tienen una relación estadísticamente significativa
               con la columna objetivo, basada en un p-valor menor o igual al umbral especificado.
    """
    # Verificar que p-valor es una probabilidad
    if not 0 < pvalue < 1:
        print("Error: El p-valor debe estar en [0,1].")
        return None
    # Verificar que target_col está en el DataFrame
    if target_col not in df.columns:
        print(f"Error: La columna '{target_col}' no se encuentra en el DataFrame.")
        return None

    # Verificar que target_col es numérica
    if not pd.api.types.is_numeric_dtype(df[target_col]):
        print(f"Error: La columna '{target_col}' debe ser numérica.")
        return None
    cardinalidad = df[target_col].nunique()
    total_filas = len(df)
    porcentaje_cardinalidad = (cardinalidad / total_filas) * 100

    # Verificar que target_col tiene cardinalidad suficiente para ser continua o discreta
    if porcentaje_cardinalidad<=10:
        print(f"Error: La columna '{target_col}' no tiene suficiente cardinalidad para ser considerada númerica continua o discreta.")
        return None

    
    # Lista para guardar las columnas categóricas significativas
    significant_cats = []
    for col in df.select_dtypes(include=['object', 'category']).columns:
        grouped = df.groupby(col, observed=True)[target_col].apply(list)
        _, p_val = stats.f_oneway(*grouped)
        if p_val <= pvalue:
            significant_cats.append(col)

    return significant_cats